In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/mnist/", one_hot=True)

Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import cm

from tqdm import tqdm

from models import modelA, modelB, modelC, modelD
from attacks import fgmt, jsma, deepfool, cw

class Tester(object):
    
    def __init__ (self, epsilon=0.3, type=2):
        
        self.mnist = mnist
        self.sess = tf.Session()
        
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=config)
    
        # Parameters
        self.total_epoch = 1000
        self.learning_rate = 5e-5
        self.batch_size = 128
        self.n_input = 28 * 28
        self.n_size = 28
        self.n_channel = 1
        self.n_noise = 10
        self.n_hidden = 128
        self.n_class = 10
        self.alpha = 0.5
        self.epsilon = epsilon
        
        self.adv_scope='naive_{}'.format(type)
        
        models = [modelA, modelB, modelC, modelD]
        
        self.conv_net = models[type]
        self.build()
        
        self.sess.run(tf.global_variables_initializer())
        
        self.checkpoint_dir = './checkpoints'
        self.adv_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.adv_scope)
        self.adv_saver = tf.train.Saver(var_list = self.adv_vars)
        self.adv_save_file_name = 'mnist_cnn_weight_type{}.ckpt'.format(type)
        
        # restore variables
        self.adv_restore()
        
    def adv_restore(self):
        self.adv_saver.restore(self.sess, os.path.join(self.checkpoint_dir, self.adv_save_file_name))
        
        
    def build(self):
        ## Modelling GAN
        # Input
        self.original_Y = tf.placeholder(tf.int32, [None], name='original_Y')
        self.Y = tf.placeholder(tf.int32, [None], name='cnn_Y')
        self.X = tf.placeholder(tf.float32, [None, self.n_input], name='test_X_')
        
        self.X_image = tf.reshape(self.X, (-1, self.n_size, self.n_size, self.n_channel))
        
        self.adv_X = jsma(self.conv_net, self.X_image, 0, eps=0.3, epochs=100)
        # self.adv_X = fgmt(self.conv_net, self.X_image, eps=0.3, epochs=10)
        self.adv_X_image = tf.reshape(self.adv_X, (-1, self.n_size, self.n_size, self.n_channel))
        self.perturb = self.adv_X_image - self.X_image
        
        self.pred = self.conv_net(self.X_image, is_training=False, scope=self.adv_scope)
        self.adv_pred = self.conv_net(self.adv_X_image, is_training=False, scope=self.adv_scope)
        
        correct_pred = tf.equal(tf.cast(tf.argmax(self.pred, 1), tf.int32), self.original_Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        adv_pred = tf.equal(tf.cast(tf.argmax(self.adv_pred, 1), tf.int32), self.original_Y)
        self.adv_accuracy = tf.reduce_mean(tf.cast(adv_pred, tf.float32))
        
    def predict(self, X):
        return np.argmax(self.sess.run(self.pred, feed_dict={self.X: X}), axis=1)
    
    def auto_test(self):
        # test mnist_cnn
        X_t = mnist.test.images[:10]
        real_Y_t = mnist.test.labels[:10]
        Y_t = self.sess.run(self.pred, {self.X: X_t})
        print(np.argmax(real_Y_t, 1))
        print(np.argmax(Y_t, 1))

        target_images = mnist.test.images[:100]
        target_labels = self.predict(target_images)
        target_Ys = (target_labels + 1) % self.n_class
        
        ones = np.ones(100)
        
        

        adv_X, perturb_X, succ = self.sess.run([self.adv_X, self.perturb, self.adv_accuracy], feed_dict={self.X: target_images, self.Y: ones, self.original_Y: target_labels})
        
        
        print(self.predict(np.reshape(adv_X, (100, -1)))[:20])
        print('succ rate: {:.3}'.format(1 - succ))
        

        example_size = 5
        fig = plt.figure(figsize=(example_size * 8, 8))

        X_it = np.arange(0, 28)
        Y_it = np.arange(0, 28)
        X_m, Y_m = np.meshgrid(X_it, Y_it)

        for i in range(example_size):
            ax = fig.add_subplot(2, example_size, i + 1)
            ax.set_axis_off()
            ax.imshow(np.reshape(adv_X[i], (28, 28)), cmap=plt.get_cmap('gray'))

        for i in range(example_size):
            ax = fig.add_subplot(2, example_size, i + example_size * 1 + 1)
            bar = ax.imshow(np.reshape(perturb_X[i], (28, 28)), cmap=cm.coolwarm)
            plt.colorbar(bar, ax=ax)


        plt.show()
        plt.close(fig)



In [3]:
sess = tf.Session()
tester = Tester(sess)
tester.auto_test()

ResourceExhaustedError: OOM when allocating tensor with shape[128]
	 [[Node: naive/fc4/biases/Assign = Assign[T=DT_FLOAT, _class=["loc:@naive/fc4/biases"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](naive/fc4/biases, naive/fc4/biases/Initializer/zeros)]]

Caused by op 'naive/fc4/biases/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mlilab/tfenv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mlilab/tfenv/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mlilab/tfenv/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/mlilab/tfenv/lib/python3.5/site-packages/zmq/eventloo